# PySpark training for data engineers
## 02. Data Ingestion

### Goal

* Create an XML file with the notebook and retrieve the content using PySpark. The output is written to a pickle to be used in the next notebook.
* Creating two CSV files with the notebook and retrieve the content using PySpark. Output is written to a single pickle.

### Highlights

* `spark.wholeTextFiles()` reads one file or directory at once.
* `rdd.saveAsPickleFile()` saves an RDD straight to a pickle that can be re-used.

### Implementation
For every Spark job, **always create a context**. This basically means you get one (or more) containers running a Spark job allocated for you.

In [1]:
from pyspark import SparkConf, SparkContext
config = SparkConf().setMaster('local')
spark = SparkContext.getOrCreate(conf=config)

Verify the Spark context has been created:

In [2]:
spark

<SparkContext master=local appName=pyspark-shell>

#### XML ingestion

In [3]:
%%file test.xml
<teststructure>
    <info>testfile</info>
    <outerlist>
        <elt>
            <subelement>One</subelement>
        </elt>
        <elt>
            <subelement>Two</subelement>
        </elt>
    </outerlist>
</teststructure>

Overwriting test.xml


Create a data object by reading the XML file into an RDD. By specifying `file://` we indicate we want to read from the local file system. Often Spark is used with Hadoop and the file is read from the Hadoop filesystem and will have an URL like `hdfs://`.

Check where we are working now to get the absolute path to the XML file:

In [4]:
!pwd

/home/jitsejan/itility/pyspark-101


In [5]:
rdd = spark.wholeTextFiles('file://///home/jitsejan/itility/pyspark-101/test.xml')

We need to call `collect()` in order to see the content of the rdd, since it is a lazy execution.

In [6]:
for file in rdd.collect():
    filename, filecontent = file
    print('Found %s' % filename)
    print('Content:\n %s' % filecontent)

Found file:/home/jitsejan/itility/pyspark-101/test.xml
Content:
 <teststructure>
    <info>testfile</info>
    <outerlist>
        <elt>
            <subelement>One</subelement>
        </elt>
        <elt>
            <subelement>Two</subelement>
        </elt>
    </outerlist>
</teststructure>


We can see that the content is indeed the same as we specified on top of this notebook. Lets use the method `saveAsPickleFile` to save the RDD to a variable on disk which we can use in the next notebook.

In [7]:
rdd.saveAsPickleFile('xml-pickle-file')

#### CSV ingestion

In [8]:
%%file csvfile01.csv
john,doe,male,32
jake,doe,male,16

Overwriting csvfile01.csv


In [9]:
%%file csvfile02.csv
jane,doe,female,31
janet,doe,female,13

Overwriting csvfile02.csv


We can use a wild card to read several files into the RDD at once.

In [10]:
csvrdd = spark.wholeTextFiles('file://///home/jitsejan/itility/pyspark-101/*.csv')

Check the files that the reader picked up:

In [11]:
for filename, filecontent in csvrdd.collect():
    print(filename)

file:/home/jitsejan/itility/pyspark-101/csvfile01.csv
file:/home/jitsejan/itility/pyspark-101/csvfile02.csv


Save the RDD to disk:

In [12]:
csvrdd.saveAsPickleFile('csv-pickle-file')